In [1]:
from PIL import ImageDraw
import pytesseract
from pdf2image import convert_from_path
import os

In [2]:
def load_pdf(pdf):
    pages = convert_from_path(pdf, 300)
    page_text=[]
    j=0
    for i, page in enumerate(pages):
        text_data = pytesseract.image_to_data(page)
        for line in text_data.split('\n'):
            if line.strip():
                parts=line.split('\t')
                if len(parts)==12:
                    text_dic={}
                    level, page_num, block_num, par_num, line_num, word_num, left, top, width, height, conf, text = parts
                    if conf != '-1' and text.strip() and text != 'text':
                        text_dic['id']=j
                        text_dic['text']=text
                        text_dic['text_info']={'level':int(level),'page_num':i+1,'block_num':int(block_num),'par_num':int(par_num),'line_num':int(line_num),'word_num':int(word_num)}
                        text_dic['position']={'left':int(left),'top':int(top),'width':int(width),'height':int(height)}
                        text_dic['conf']=float(conf)
                        page_text.append(text_dic)
                        j+=1
    return page_text, pages

In [3]:
def find_box(entry):
    bottom_y=entry['position']['top']
    right_x=entry['position']['left']+entry['position']['width']
    left_x=entry['position']['left']
    top_y=entry['position']['top']+entry['position']['height']
    line = entry['text_info']['line_num']
    page=entry['text_info']['page_num']
    area=[left_x,bottom_y,right_x,top_y]
    return area, page, line

In [4]:
def close_y(entry,page_text,up=0):
    min_dist=100000
    key_area,key_page, key_line=find_box(entry)
    redactee=0
    for x in page_text:
        target_area,target_page, target_line=find_box(x)
        y_dist=key_area[1]-target_area[1]
        if not up:
            if y_dist < 0 and abs(y_dist) < min_dist and key_page == target_page  and key_line + 1 <= target_line <= key_line + 3 and (key_area[0]-100< target_area[0] < key_area[2]+100 or key_area[0]-100 < target_area[2] < key_area[2]+100):
                winner_area=target_area
                redactee=x
                min_dist=y_dist
        else:
            if y_dist > 0 and abs(y_dist) < min_dist and key_page == target_page  and key_line + 1 <= target_line <= key_line + 3 and (key_area[0]-100< target_area[0] < key_area[2]+100 or key_area[0]-100 < target_area[2] < key_area[2]+100):
                winner_area=target_area
                redactee=x
                min_dist=y_dist
    if redactee:
        return redactee, winner_area, min_dist
    else:
        return 0, 0, 0


In [5]:
def redact(targets,pages,page_text,redacted_pages,target_consistent):
    for i, page in enumerate(pages):
        print(redacted_pages)
        redacted_page=page.copy()
        draw=ImageDraw.Draw(redacted_page)
        for target in targets:
            filtered_list = [d for d in page_text if d['text'] == target]
            for entry in filtered_list:
                    if entry['text_info']['page_num']==i+1:
                        redactee, neighbor, y_dist=close_y(entry,page_text)
                        if redactee:
                            if y_dist:
                                line_num=redactee['text_info']['line_num']
                                block_num=redactee['text_info']['block_num']
                                par_num=redactee['text_info']['par_num']
                                page_num=redactee['text_info']['page_num']
                                filtered_list2 = [d for d in page_text if d['text_info']['line_num'] == line_num]
                                filtered_list2 = [d for d in filtered_list2 if d['text_info']['block_num'] == block_num]
                                filtered_list2 = [d for d in filtered_list2 if d['text_info']['par_num'] == par_num]
                                filtered_list2 = [d for d in filtered_list2 if d['text_info']['page_num'] == page_num]
                                try:    
                                    left_bound=min(filtered_list2, key=lambda x:x['position']['left'])
                                    right_bound=max(filtered_list2, key=lambda x:x['position']['left'])
                                    left_area, left_page, left_line=find_box(left_bound)
                                    right_area, right_page, right_line=find_box(right_bound)
                                    redact_area=[left_area[0],left_area[1]-0,right_area[2],right_area[3]]
                                    draw.rectangle(redact_area,fill='black')
                                    #print('Redacted! y_dist: ',y_dist,'text: ', redactee['text'], 'true page: ', i+1, 'page: ',entry['text_info']['page_num'])
                                except:
                                    print('Empty Line')
        for target in target_consistent:
            filtered_list = [d for d in page_text if d['text'] == target]
            for entry in filtered_list:
                    if entry['text_info']['page_num']==i+1:
                        redactee, winner_area, y_dist=close_y(entry,page_text)
                        if redactee:
                            redact_area=[winner_area[0]-50,winner_area[1]-0,winner_area[2]+250,winner_area[3]]
                            draw.rectangle(redact_area,fill='black')
                            #print('Redacted! y_dist: ',y_dist,'text: ', redactee['text'], 'true page: ', i+1, 'page: ',entry['text_info']['page_num'])
        redacted_pages.append(redacted_page)
        print(redacted_pages)
    return redacted_pages

In [6]:

pages = convert_from_path('test/Allen_Carole_2003.pdf', 300)
page_text=[]
j=0
for i, page in enumerate(pages):
    text_data = pytesseract.image_to_data(page)
    for line in text_data.split('\n'):
        if line.strip():
            parts=line.split('\t')
            if len(parts)==12:
                text_dic={}
                level, page_num, block_num, par_num, line_num, word_num, left, top, width, height, conf, text = parts
                if conf != '-1' and text.strip() and text != 'text':
                    text_dic['id']=j
                    text_dic['text']=text
                    text_dic['text_info']={'level':int(level),'page_num':i+1,'block_num':int(block_num),'par_num':int(par_num),'line_num':int(line_num),'word_num':int(word_num)}
                    text_dic['position']={'left':int(left),'top':int(top),'width':int(width),'height':int(height)}
                    text_dic['conf']=float(conf)
                    page_text.append(text_dic)
                    j+=1

targets = ['ADDRESS']
target_consistent = ['CODE/PHONE']
output_list = []
print(output_list)
output_list=redact(targets, pages, page_text, output_list,target_consistent)
print(output_list)

[]
[]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC95B6E550>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC95B6E550>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC95B6E550>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC951EF510>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC96818650>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC95B6E550>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x1CC951EF510>]


In [14]:
import os

directory = 'test'

for filename in os.listdir(directory):
    if filename.endswith('.pdf'):
        file_path = os.path.join(directory, filename)
        output = filename[:-4]  # More Pythonic way to remove file extension
        page_text, pages = load_pdf(f'test/{filename}')
        targets = ['ADDRESS']
        target_consistent = ['CODE/PHONE']
        redacted_pages = []
        redacted_pages=redact(targets, pages, page_text, redacted_pages,target_consistent)

        # Proceed only if there are redacted pages
        if redacted_pages:
            imgs = [i.convert('RGB') for i in redacted_pages]
            # Check if there's at least one image to save
            if imgs:
                imgs[0].save(f'output/{output}_redacted.pdf', 'PDF', resolution=100.0, save_all=True, append_images=imgs[1:])
            else:
                print(f"No pages to save for {filename}")
        else:
            print(f"No redacted pages for {filename}")


[]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E1862450>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E1862450>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E1862450>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E0E38DD0>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E1862450>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E0E38DD0>]
[<PIL.Image.Image image mode=RGB size=3300x2550 at 0x1938108B690>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E1862450>, <PIL.Image.Image image mode=RGB size=3300x2550 at 0x193E0E38DD0>, 